In [1]:
import pandas as pd
import numpy as np
import csv

#크롤링을 위한 라이브러리
import requests
from bs4 import BeautifulSoup 
import re
import urllib.request
import time
from time import sleep

from selenium.webdriver import Chrome   
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium import webdriver

In [2]:
# 가져올 사이트 페이지 URL
url ='https://thinkyou.co.kr/contest/sector.asp'

In [3]:
# 크롬 드라이버 위치 지정 (상대경로 아니니까 각자 PC에 맞게 알아서 경로 잡아랏)
browser = Chrome(r"C:\Users\User\Downloads\chromedriver_win32 (1)\chromedriver.exe")

# 모든 자원이 로드될때까지 기다리게하는 시간 지정 (3초)
delay=3
browser.implicitly_wait(delay)

In [4]:
# get()을 통해 해당 URL에 접근
browser.get(url) 

# 크롬 브라우저 창 크기 조절
browser.maximize_window()

In [7]:
# tag name 통해 element에 접근
body = browser.find_element_by_tag_name('body')

In [9]:
# 과학/공학 카테고리 글만 보기
try :
    # xpath 통해 element 가져오기
    # F12하고 과학/공학 버튼 위치 간다음 우클릭 copy 하면 copy xpath 뜸
    # 해당 xpath : //*[@id="searchFrm"]/div/dl[1]/dd/p[6]/label/span
    browser.find_elements_by_xpath('//*[@id="searchFrm"]/div/dl[1]/dd/p[6]/label/span')[0].click()
except:
    pass

In [10]:
# 웹페이지 스크롤
# https://cnpnote.tistory.com/entry/PYTHON-%ED%8C%8C%EC%9D%B4%EC%8D%AC%EC%97%90%EC%84%9C-selenium-webdriver%EB%A5%BC-%EC%82%AC%EC%9A%A9%ED%95%98%EC%97%AC-%EC%9B%B9-%ED%8E%98%EC%9D%B4%EC%A7%80%EB%A5%BC-%EC%8A%A4%ED%81%AC%EB%A1%A4%ED%95%98%EB%8A%94-%EB%B0%A9%EB%B2%95%EC%9D%80-%EB%AC%B4%EC%97%87%EC%9E%85%EB%8B%88%EA%B9%8C
# 실행하고 나면 해당 페이지 스크롤 맨아래까지 내려감

SCROLL_PAUSE_TIME = 0.5
while True:
    last_height = browser.execute_script('return document.documentElement.scrollHeight')
    # 현재 화면의 길이를 리턴 받아 last_height에 넣음
    for i in range(3):
        body.send_keys(Keys.END)
        # body 본문에 END키를 입력(스크롤내림)
        time.sleep(SCROLL_PAUSE_TIME)
    new_height = browser.execute_script('return document.documentElement.scrollHeight')
    if new_height == last_height:
        break;

In [13]:
body

<selenium.webdriver.remote.webelement.WebElement (session="8664f8565e8d711ee51d733f9d604a2e", element="531a4588-a8f4-41fa-b6f3-e64a3c2cf5e6")>

In [19]:
# 페이지 소스 리턴
# https://beomi.github.io/2017/02/27/HowToMakeWebCrawler-With-Selenium/

page = browser.page_source #페이지 Element모두 가져옴
soup = BeautifulSoup(page, 'lxml') # Beautiful Soup 사용하기

In [22]:
page

'<html lang="ko" class="sub"><head>\n\t<meta charset="utf-8">\n\t<meta http-equiv="X-UA-Compatible" content="IE=edge">\n\n\t<!-- Global site tag (gtag.js) - Google Analytics -->\n\t<script type="text/javascript" async="" src="https://www.google-analytics.com/analytics.js"></script><script async="" src="https://www.googletagmanager.com/gtag/js?id=UA-163306206-1"></script>\n\t<script>\n\t  window.dataLayer = window.dataLayer || [];\n\t  function gtag(){dataLayer.push(arguments);}\n\t  gtag(\'js\', new Date());\n\n\t  gtag(\'config\', \'UA-163306206-1\');\n\t</script>\n\t<!-- Global site tag (gtag.js) - Google Analytics -->\n\n\t\n\t<meta http-equiv="X-UA-Compatible" content="IE=edge">\n\t<meta property="og:type" content="website">\n\t<meta property="og:title" content="공모전, 대외활동|씽유">\n\t<meta property="og:url" content="https://thinkyou.co.kr/">\n\t<meta property="og:description" content="공모전 대외활동 서포터즈 대학생대외활동 공모전사이트 공모전정보 대학생공모전 사진공모전 디자인공모전 광고공모전 영상공모전">\n\t<meta property="og:image" cont

In [23]:
soup

<html class="sub" lang="ko"><head>
<meta charset="utf-8"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<!-- Global site tag (gtag.js) - Google Analytics -->
<script async="" src="https://www.google-analytics.com/analytics.js" type="text/javascript"></script><script async="" src="https://www.googletagmanager.com/gtag/js?id=UA-163306206-1"></script>
<script>
	  window.dataLayer = window.dataLayer || [];
	  function gtag(){dataLayer.push(arguments);}
	  gtag('js', new Date());

	  gtag('config', 'UA-163306206-1');
	</script>
<!-- Global site tag (gtag.js) - Google Analytics -->
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta content="website" property="og:type"/>
<meta content="공모전, 대외활동|씽유" property="og:title"/>
<meta content="https://thinkyou.co.kr/" property="og:url"/>
<meta content="공모전 대외활동 서포터즈 대학생대외활동 공모전사이트 공모전정보 대학생공모전 사진공모전 디자인공모전 광고공모전 영상공모전" property="og:description"/>
<meta content="https://thinkyou.co.kr/images/sns_link.png" property="og:image"/>
<met

In [43]:
links = [] # 링크
titles = [] # 행사 이름
dday = [] # D-DAY
inst = [] # 주최 측

In [25]:
#길이
# soup.select(내가 원하는 조건) : 마감이라 써져있으면 안 가져오게
# 이 사이트의 div class = "title" / title 앞엔 .을, id title이면 앞에 #을 붙임
for i in range(len(soup.select(' .title > a > dl > dt '))):
    if soup.select(' .statNew > p ')[i].text == '마감':
        break

In [36]:
soup.select(' a > dl > dt ')[i].text

'2021 ICT 융합 프로젝트 공모전 '

In [26]:
# 마감 전까지의 공모전 개수
len_day = i

In [54]:
soup.select(' .title > a > dl > dd ')[0].text.split(':')[1][1:]

'대구광역시 스마트시티지원센터'

In [44]:
#데이터 수집
for i in range(len_day):
    base_url = 'https://thinkyou.co.kr'
    titles.append(soup.select(' .title > a > dl > dt ')[i].text)
    inst.append(soup.select(' .title > a > dl > dd ')[i].text.split(':')[1][1:])
    dday.append('D' + soup.select(' .statNew')[i].text.split('D')[1])
    links.append(base_url + soup.select(' .title > a')[i]['href'][2:])

In [45]:
tabl_data = {'제목': titles, 
            '날짜': dday,
            '기관':inst,
            '링크':links}

In [46]:
df = pd.DataFrame(tabl_data, columns=['제목','날짜','기관','링크'])  

In [60]:
df

제목     날짜               기관  \
0         2021 SMART CITY 시민참여 아이디어 경진대회      D-7  대구광역시 스마트시티지원센터   
1                    2021 ICT 융합 프로젝트 공모전    D-10   (주)엔티렉스-디바이스마트   
2       2020 기계 시설물 분야 AI 학습용 데이터 활용 경진대회     D-5     NIA 한국정보화진흥원   
3                          MZ 인공지능 해커톤 대회     D-2      한국지능정보사회진흥원   
4                          2020 DACON CUP   D-day              데이콘   
5              군산대학교 해상풍력연구원 모니터링 하우스 공모전   D-330      군산대학교 산학협력단   
6  2020 ZEISS Microscopy Research Contest    D-37            ZEISS   

                                                  링크  
0  https://thinkyou.co.kr/contest/sector_view.asp...  
1  https://thinkyou.co.kr/contest/sector_view.asp...  
2  https://thinkyou.co.kr/contest/sector_view.asp...  
3  https://thinkyou.co.kr/contest/sector_view.asp...  
4  https://thinkyou.co.kr/contest/sector_view.asp...  
5  https://thinkyou.co.kr/contest/sector_view.asp...  
6  https://thinkyou.co.kr/contest/sector_view.asp...

In [19]:
df.to_csv('thinkyou.csv', encoding='utf-8-sig')